In [1]:
# Using ML to classify LAEs in the NEP Field
import tables as tb
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from astropy import constants as const
from astropy.table import Table, column, join
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import ZScaleInterval

from regions import CircleSkyRegion, CirclePixelRegion

from hetdex_api.survey import Survey, FiberIndex
from hetdex_api.config import HDRconfig
from hetdex_api.detections import Detections

import pandas as pd
import seaborn as sb

In [2]:
config = HDRconfig(survey = 'hdr2.1')
file_elix = tb.open_file(config.elixerh5)

In [3]:
#getting the detections info from the h5 file
Detections_h5 = file_elix.root.Detections

In [ ]:
type(Detections_h5)

In [ ]:
#convert everything in the Detections h5 into a DataFrame

#function that returns everything in the detections h5 file as a DataFrame
def Detections_DF(det_h5):
    h5_dictionary = {}
    for cols in det_h5.colnames:
        h5_dictionary[cols] = det_h5.col(cols)
    
    Detection_DF = pd.DataFrame(h5_dictionary)
    Detection_DF = Detection_DF.set_index('detectid')
    
    return Detection_DF

In [ ]:
# This may take some time to load since it is reading in all the millions of Spectra in HDR2.1
# but this will give you ALL the detections in HDR2.1 - Oscar

Total_Detections_DF = Detections_DF(Detections_h5)

In [4]:
det_object = Detections('hdr2.1', loadtable=False)

In [5]:
# Once it has loaded you want to filter out the data by selecting those that are in the NEP field
# to do this I will give you the verticies of a box that will encompass all the NEP field - Oscar

# The center of the NEP field is given by:
# NEP Central Coordinates:
# R.A. = 18hours00minutes00seconds, decl. = 66 degree 33minute 38.552 arcmin
# Then make a radius of 3.5 degrees centered above and find all the RA and DEC coordinates
# in the DF that are within this circle

########################
#NEP_RA_filter = (Total_Detections_DF['ra'] > 266.5) & (Total_Detections_DF['ra'] < 273.5) & (Total_Detections_DF['dec'] > 63.05) & (Total_Detections_DF['dec'] < 70.05)

# creating the circle region in the sky (NEP field)
#ra = '18h00m00s'
#dec = '+66d33m38.552s'
#center_sky_coords = SkyCoord(ra, dec, frame = 'icrs')
#circle_region_sky = CircleSkyRegion(center_sky_coords, radius = 3.5 * u.deg)

ra = '18h00m00s'
dec = '+66d33m38.552s'
center_sky_coords = SkyCoord(ra, dec, frame = 'icrs')
maskregion = det_object.query_by_coords(center_sky_coords, 3.5 * u.deg)
detects_in_region = det_object[maskregion]
print(np.size(detects_in_region.detectid))
# 
########################
#circle_region_sky

69799


In [ ]:
#circle_region_sky

In [ ]:
# Once you have selected sources within the NEP footprint we can then go ahead and find some
# spectra from these sources - Oscar
Filtered_Detections = Total_Detections_DF.loc[NEP_RA_filter, ['ra', 'dec']]

In [ ]:
spec = det_object.get_spectrum(2101274049)

In [ ]:
spec

center_ksy.separations(skycoords entire)
    return indeces
    return distance
    return 3d

separate by dist
mask

main goal of algorithm
want it to distinguish lae vs o2 emitter. wouldn't impose cuts unless training.

cut = filter
cut = signal to noise could do plya

might need cuts for taining for confident lya and o2

increase confidence by visually inspecting

could visually inspect to increase confidence.

plotting histograms to look for outliers

hetdex isn't perfect and it catches emission lines that aren't real. visual inspections helps

no need for dataframes if i found another way